In [ ]:
import pandas as pd



# Variables

filename = r'C:\Users\USER\Desktop\DESPEGAR\fb_vehiculos_27122022_2138 (1).csv'

sufix = 'corregido'

filenameFixed = filename.split('.')[0] +'-' + sufix + '.'+ filename.split('.')[1]



# Lectura del archivo csv

data = pd.read_csv(filename, sep=';', encoding='utf-8')



# Cambio de codificación a latin-1 por columnas específicas (solo las de texto)

cols = ['titulo_marketplace', 'descripcion', 'vendedor', 'locacion']

data[cols] = data[cols].apply(lambda x: x.str.encode('latin-1', 'ignore').str.decode('latin-1'))   



# Lectura del archivo csv corregido

data.to_csv(filenameFixed, sep=';', index=False, encoding='latin-1')